# Final Project: Overview

# Objective

The objective of this project is for you to demonstrate your mastery of the Machine Learning process
**using Neural Networks**.



# Submission requirements

The guidelines will be similar to the Midterm
- you will write a procedure that takes raw data and produces predictions

You will submit a *single* model for evaluation.

**Demonstrate that all cells in your notebook work**

The final cell in your notebook should print the message "Done"
- `print("Done")`
- If we run your notebook and this last cell does not execute your submission will be inadequate

## Testing

*You must perform out of sample testing*.

If you want to perform cross-validation in training, that is fine, but you
must *also* test out of sample to show that you are not over-fitting.

It is up to you to create the out of sample data that you feel best evaluates your model.

We will create holdout data (that we will not show you) for grading.

The procedure you write to make predictions should be able to work on the unseen holdout data
(similar to how it should work for your test set but the holdout set has *no targets*)

    

# The data

Data will be provided to you
- as multiple files in a directory which we refer to as a *data directory*

The reason for this is that the different files may convey different information.

You will be responsible for deciding
- which files to use
- which fields within the files to use

We will give you a data directory for training.

# Submission guidelines

Here are the basics, a code template that you must complete is in the following cells
- you will be required to store  your model in a file
- you will be required to write a procedure `MyModel` that takes two arguments
    - `test_dir`
        - this is a *relative path* to the holdout data directory
    - `model_save_file`
        - this is a *relative path* to the file containing your model
- the holdout data directory is similar in structure to the training data directory
    - but without target labels !  It is your job to predict these.
- your procedure must produce predictions given this holdout data directory

This means that your procedure must
- prepare the files in the holdout data directory similar to the way that they were prepared in the training data directory

We will provide you with a sample data directory that will resemble the holdout -- this is so that you
may test the procedure you write for submission.



## Detailed submission guidelines


In **addition to your notebook that trains/evaluates your model**,
- please also submit the file that contains your trained model.
    - use `saveModel` to put your final, trained model in this file
- We will **not** train your model; we will only use the method `MyModel`
    - which **you** will implement
    - and which uses `loadModel` and the name of the file in which the model was saved
    - this will create the model that we will evaluate


Here is a code template for you to complete
- it will save your model (assuming it is in variable `my_model`)
- it provides the specification for procedure `MyModel`, which *you must complete*


In [ ]:
import os
from tensorflow.keras.models import load_model

modelName = "final_model"
model_path = os.path.join(".", modelName)

def saveModel(model, model_path):
    """
    Save a model

    Parameters
    ----------
    model: a Keras model object
    model_path: String.  Relative path to a file in which the model will be stored


    Returns
    -------
    model_save_file: String.  Name of the file in which the model has been saved
        - this will be model_path with the proper file-extension added
    """

    model_save_file = model_path + '.keras'

    model.save(model_save_file)

    print(f"Model saved in {model_save_file:s}; submit with your assignment.")

    return model_save_file

def loadModel(model_save_file):
    model = tf.keras.models.load_model( model_save_file )

    return model

def MyModel(test_dir, model_save_file):
    # YOU MAY NOT change model after this statement !
    model = loadModel(model_save_file)

    # It should run model to create an array of predictions; we initialize it to the empty array for convenience
    predictions = []

    # We need to match your array of predictions with the examples you are predicting
    # The array below (ids) should have a one-to-one correspondence and identify the example your are predicting
    # For Bankruptcy: the Id column
    # For Stock prediction: the date on which you are making a prediction
    ids = []

    # YOUR CODE GOES HERE


    return predictions, ids

# Assign to variable my_model the model that is your final model (the one  you will be evaluated on)
my_model = None # CHANGE None to your model !

# model_save_file = saveModel(my_model, model_path)

# 0. Constants

In [ ]:
from typing import Final

DATASET_URL: Final = "https://github.com/rogerli2020/Stock-Prediction-FRE7773-Final-Project/raw/refs/heads/main/Data.tar"
TRAIN_SET_RELATIVE_PATH: Final = "./Data/train"
SAMPLE_SET_RELATIVE_PATH: Final = "./Data/sample"

# 1. Getting the Data

In [ ]:
import os
import urllib.request
import tarfile
import pandas as pd

class MyDataset:
  def __init__(self) -> None:
    self.train_dataset = None
    self.sample_dataset = None

    self._initialize_dataset()

  def _initialize_dataset(self) -> None:
    def load_dataset_from_dir(dir : str) -> dict:
      data_dict = {}
      if not os.path.isdir(dir):
        raise ValueError(f"The provided path '{dir}' is not a directory or does not exist.")
      for file_name in os.listdir(dir):
        if file_name.endswith(".csv"):
            file_path = os.path.join(dir, file_name)
            key = os.path.splitext(file_name)[0]
            try:
                data_dict[key] = pd.read_csv(file_path)
            except Exception as e:
                print(f"Error reading {file_name}: {e}")
      return data_dict

    print("Initializing dataset...")
    self._download_dataset()
    self.train_dataset = load_dataset_from_dir(TRAIN_SET_RELATIVE_PATH)
    self.sample_dataset = load_dataset_from_dir(SAMPLE_SET_RELATIVE_PATH)
    assert isinstance(self.train_set, dict) and len(self.train_set) > 0, "train_set not intialized correctly"
    assert isinstance(self.sample_set, dict) and len(self.sample_set) > 0, "sample_set not initialized correctly"
    print("Dataset initialized.")


  def _download_dataset(self):
    try:
      tar_file_path, _ = urllib.request.urlretrieve(DATASET_URL)
      with tarfile.open(tar_file_path, "r") as tar:
          tar.extractall()
    except Exception as e:
      print(f"An error occurred during download or extraction: {e}")
    finally:
      if os.path.exists(tar_file_path):
          os.remove(tar_file_path)

dataset = MyDataset()

Download completed. File saved as /tmp/tmpu74g76yn.
Extracting /tmp/tmpu74g76yn to tmpu74g76yn...
Extraction completed. Dataset available in tmpu74g76yn.


## Evaluate your model on the holdout data directory

**You must run the following cell** from the directory that contains your model file

Here is how we will evaluate your submission
- we will create a directory whose only content is
    - sub-directory `Data`
- we will restore your model from the saved file`
- we will run the cell in your notebook that should be a copy of the one below
    - it calls procedure `MyModel` with the arguments given below
    - your implementation of `MyModel`
        - must successfully load your model file, *given where **we** have place it as described above*
        - must successfully return one prediction for each example in the holdout directory *given where **we** have placed the holdout directory*

In [ ]:
holdout_dir = os.path.join(".", "Data", "holdout")
predicts = MyModel(holdout_dir, model_save_file)

In [ ]:
print("Done")